
# **Demo: Revolutionizing Product-Specific AI with Fireworks Multi-LoRA**

## **Overview**

This demo highlights the capabilities of **Firework's Multi-LoRA**, a cutting-edge technique for **fine-tuning models across various product domains**. By tailoring the AI to answer product-related inquiries in specific categories like beauty, fashion, outdoor products, and baby products, Multi-LoRA ensures more precise, relevant responses. This eliminates the need for users to rely on generic filters and enables them to ask product-specific questions directly, with the AI providing answers based on domain-specific knowledge.

For instance, a customer looking for a specific beauty product, fashion item, or outdoor gear can simply ask an AI Chatbot without having to sort through broad categories or apply generic filters.

## **Datasets Used**

* **Beauty and Personal Care:** [Amazon Beauty and Personal Care 2023](https://huggingface.co/datasets/smartcat/Amazon_Beauty_and_Personal_Care_2023)
* **Fashion:** [Amazon Clothing, Shoes, and Jewelry 2023](https://huggingface.co/datasets/smartcat/Amazon_Clothing_Shoes_and_Jewelry_2023)
* **Outdoor:** [Amazon Sports and Outdoors 2023](https://huggingface.co/datasets/smartcat/Amazon_Sports_and_Outdoors_2023)
* **Baby Products:** [Amazon Baby Products 2023](https://huggingface.co/datasets/smartcat/Amazon_Baby_Products_2023)

## **Steps in the Demo**

0. **General Setup**
1. **Prepare Dataset**
  *   **Download Data:** Gather the necessary data for fine-tuning.
  *   **Create Dataset Records:** Register the datasets in the system.
  *   **Upload Datasets:** Upload the datasets to the platform.
  *   **Validate Dataset Uploads:** Ensure the datasets are ready for fine-tuning.
2. **Prepare Fine-Tuning Job**
  * **Create Fine-Tuning Jobs:** Define and execute fine-tuning jobs tailored to your datasets.
  * **Get Fine-Tuning Job IDs:** Retrieve unique identifiers for the fine-tuning jobs to clean up at the end.
  * **Validate Fine-Tuning Jobs:** Verify the completion and accuracy of the fine-tuning tasks.
3. **Deploy and Use Fine-Tuned Model for Inference**
  * **Access and Download Base Model:** Retrieve the base model to work with.
  * **Obtain LoRA Adapters:** Get the necessary LoRA adapters for the model.
  * **Merge Base Model with LoRA Adapters:** Integrate the LoRA adapters into the base model.
  * **Upload and Deploy Merged Model:** Upload and deploy the newly merged model.
  * **Inference:** Use the deployed model to generate predictions or perform tasks.
4. **Clean Up**
  * **Undeploy and Delete Model:** Remove the deployed model once it's no longer needed.
  * **Delete Fine-Tuning Jobs:** Clear fine-tuning job records to free up resources.
  * **Delete Datasets:** Remove datasets to maintain a tidy workspace.

This demo demonstrates how Multi-LoRA can deliver personalized product information without relying on broad, generic filters. Users can now ask questions directly related to a product category, enabling a smoother and more efficient experience.

## **Important Note**

The demo leverages both the Fireworks AI REST API and Fireworks' command-line utility, `firectl`. REST API support for steps currently handled via the `firectl` command-line utility will be added soon.

# **General Setup**

In [4]:
!pip install datasets
!pip install fireworks-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 8.1 MB/s eta 0:00:00
  At

In [5]:
from google.colab import drive
from datasets import load_dataset
import requests
import os
import json

In [7]:
# Fireworks API configurations
API_KEY = "YOUR_API_KEY"
ACCOUNT_ID = "YOUR_ACCOUNT_ID"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
HEADERS_WITH_CONTENT_TYPE = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

BASE_URL = f"https://api.fireworks.ai/v1/accounts/{ACCOUNT_ID}"
BASE_MODEL_ID = "llama-v3p2-1b-instruct" # YOUR_BASE_MODEL_ID
DEPLOYMENT_ID = "mutlilora-demo-test-model"

# Dataset URLs and names
DATASETS = {
    "care": "smartcat/Amazon_Beauty_and_Personal_Care_2023",
    "fashion": "smartcat/Amazon_Clothing_Shoes_and_Jewelry_2023",
    "outdoor": "smartcat/Amazon_Sports_and_Outdoors_2023",
    "baby": "smartcat/Amazon_Baby_Products_2023"
}

# Dataset IDs and names
DATASET_IDS = {
    "care": "care-multilora-demo-test-data",
    "fashion": "fashion-multilora-demo-test-data",
    "outdoor": "outdoor-multilora-demo-test-data",
    "baby": "baby-multilora-demo-test-data"
}

# Model IDs and names
MODEL_IDS = {
    "care": "care-multilora-demo-test-model",
    "fashion": "fashion-multilora-demo-test-model",
    "outdoor": "outdoor-multilora-demo-test-model",
    "baby": "baby-multilora-demo-test-model"
}

# Fine-tuning Jobs IDs and names
# Note: updated appropriately as new fine-tuning jobs are created!
fine_tuning_jobs = {
    "care": "",
    "fashion": "",
    "outdoor": "",
    "baby": ""
}

# **Prepare Datasets**

## Download data

In [7]:
# Mount Google Drive to access and store files
drive.mount('/content/drive', force_remount=True)

# Define a directory in Google Drive to store datasets
DATA_DIR = "/content/drive/MyDrive/fireworks_datasets"
os.makedirs(DATA_DIR, exist_ok=True)  # Ensure the directory exists

def download_and_prepare_data(dataset_name):
    """
    Download dataset from Hugging Face and save it as a JSONL file in Google Drive.

    Args:
        dataset_name (str): Name of the dataset to download.

    Returns:
        str: Local path to the saved JSONL file.
    """
    # Load dataset from Hugging Face Hub
    dataset = load_dataset(DATASETS[dataset_name])

    # Note: Reducing the dataset size is not necessary.
    # Large datasets can be uploaded using the Fireworks API:
    # https://docs.fireworks.ai/api-reference/get-dataset-upload-endpoint

    # Select 100000 enties for each dataset
    dataset["train"] = dataset["train"].select(range(10000))

    # Define the local path to save the dataset
    local_path = os.path.join(DATA_DIR, f"{dataset_name.split('/')[-1]}.jsonl")

    # Save the dataset's train split as a JSONL file
    dataset["train"].to_json(local_path)

    print(f"Dataset saved at {local_path}")
    return local_path

Mounted at /content/drive


## Create Dataset Records

In [8]:
def create_dataset_record_on_fireworks(dataset_name):
    """
    Create a dataset record on the Fireworks platform.

    Args:
        dataset_name (str): Name of the dataset to register on Fireworks.
    """
    # Retrieve the unique dataset ID associated with the given dataset name
    dataset_Id = DATASET_IDS[dataset_name]

    # Payload to send to the API
    payload = {
        "datasetId": dataset_Id,
        "dataset": {
            "userUploaded": {}
        }
    }

    # API endpoint URL for creating datasets
    url = f"{BASE_URL}/datasets"

    # Make a POST request to the Fireworks API
    response = requests.post(url, headers=HEADERS_WITH_CONTENT_TYPE, json=payload)

    # Handle response
    if response.status_code == 200:
      print(f"Created dataset {dataset_Id} record successfully.")
      print(response.json())
    else:
      print(f"Failed to create dataset {dataset_Id} record: {response.status_code}")
      print(response.text)
      response.raise_for_status()

In [9]:
# Loop through the datasets and process each one
for dataset_name in DATASETS:

    # Create dataset record on Fireworks
    create_dataset_record_on_fireworks(dataset_name)

Created dataset care-multilora-demo-test-data record successfully.
{'createTime': '2025-01-06T22:04:41.394929376Z', 'displayName': '', 'exampleCount': '0', 'format': 'FORMAT_UNSPECIFIED', 'name': 'accounts/fireworks/datasets/care-multilora-demo-test-data', 'state': 'UPLOADING', 'status': {'code': 'OK', 'message': ''}, 'userUploaded': {}}
Created dataset fashion-multilora-demo-test-data record successfully.
{'createTime': '2025-01-06T22:04:41.746275567Z', 'displayName': '', 'exampleCount': '0', 'format': 'FORMAT_UNSPECIFIED', 'name': 'accounts/fireworks/datasets/fashion-multilora-demo-test-data', 'state': 'UPLOADING', 'status': {'code': 'OK', 'message': ''}, 'userUploaded': {}}
Created dataset outdoor-multilora-demo-test-data record successfully.
{'createTime': '2025-01-06T22:04:42.072962076Z', 'displayName': '', 'exampleCount': '0', 'format': 'FORMAT_UNSPECIFIED', 'name': 'accounts/fireworks/datasets/outdoor-multilora-demo-test-data', 'state': 'UPLOADING', 'status': {'code': 'OK', 'mes

## Upload Datasets

In [10]:
def upload_data_to_fireworks(local_path, dataset_name):
    """
    Upload the dataset file to Fireworks using multipart/form-data.

    Args:
        local_path (str): Path to the dataset file on the local filesystem.
        dataset_name (str): Name of the dataset to upload.
    """
    # Retrieve the unique dataset ID associated with the given dataset name
    dataset_Id = DATASET_IDS[dataset_name]

    # API endpoint URL for uploading datasets
    url = f"{BASE_URL}/datasets/{dataset_Id}:upload"

    # Open the dataset file in binary read mode
    with open(local_path, "rb") as f:
        # Define the file payload for the multipart/form-data request
        files = {
            "file": f  # File content
        }

        # Make a POST request to upload the file
        response = requests.post(url, headers=HEADERS, files=files)

        # Handle the API response
        if response.status_code == 200:
            print(f"Dataset {dataset_Id} uploaded successfully.")
            print(response.json())
        else:
            print(f"Failed to upload dataset {dataset_Id}: {response.status_code}")
            print(response.text)

In [11]:
# Loop through the datasets and process each one
for dataset_name in DATASETS:
    # Download and prepare data
    local_path = download_and_prepare_data(dataset_name)

    # Upload the dataset to Fireworks
    upload_data_to_fireworks(local_path, dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/209M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/191M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/7.67k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/33.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22767 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Dataset saved at /content/drive/MyDrive/fireworks_datasets/baby.jsonl
Dataset baby-multilora-demo-test-data uploaded successfully.
{'bytes': 41046825, 'created_at': 1736201082, 'filename': 'baby.jsonl', 'id': 'accounts/fireworks/datasets/baby-multilora-demo-test-data', 'object': 'file', 'purpose': 'dataset'}


## Validate Dataset Uploads

In [12]:
def validate_dataset_upload(dataset_name):
    """
    Validate the uploaded dataset on Fireworks.

    Args:
        dataset_name (str): Name of the dataset to validate.
    """
    # Retrieve the unique dataset ID associated with the given dataset name
    dataset_Id = DATASET_IDS[dataset_name]

    # Payload to send to the API
    payload = {}

    # Define the validation URL
    url = f"{BASE_URL}/datasets/{dataset_Id}"

    # Make the GET request to validate the dataset
    response = requests.get(url, headers=HEADERS_WITH_CONTENT_TYPE)

    # Handle the API response
    if response.status_code == 200:
        # Check that the status is "READY"
        dataset_dict = json.loads(response.text)
        state = dataset_dict["state"]
        print(f"Dataset {dataset_Id} is {state}.")
        print(response.json())
    else:
        print(f"Failed to upload dataset {dataset_Id}: {response.status_code}")
        print(response.text)

In [13]:
# Loop through the datasets and process each one
for dataset_name in DATASETS:

  # Validate uploaded dataset
  validate_dataset_upload(dataset_name)

Dataset care-multilora-demo-test-data is READY.
{'createTime': '2025-01-06T22:04:41.394929Z', 'displayName': '', 'exampleCount': '0', 'format': 'FORMAT_UNSPECIFIED', 'name': 'accounts/fireworks/datasets/care-multilora-demo-test-data', 'state': 'READY', 'status': {'code': 'OK', 'message': ''}, 'userUploaded': {}}
Dataset fashion-multilora-demo-test-data is READY.
{'createTime': '2025-01-06T22:04:41.746275Z', 'displayName': '', 'exampleCount': '0', 'format': 'FORMAT_UNSPECIFIED', 'name': 'accounts/fireworks/datasets/fashion-multilora-demo-test-data', 'state': 'READY', 'status': {'code': 'OK', 'message': ''}, 'userUploaded': {}}
Dataset outdoor-multilora-demo-test-data is READY.
{'createTime': '2025-01-06T22:04:42.072962Z', 'displayName': '', 'exampleCount': '0', 'format': 'FORMAT_UNSPECIFIED', 'name': 'accounts/fireworks/datasets/outdoor-multilora-demo-test-data', 'state': 'READY', 'status': {'code': 'OK', 'message': ''}, 'userUploaded': {}}
Dataset baby-multilora-demo-test-data is READY

# **Prepare Fine-Tuning Jobs**




## Create Fine-Tuning Jobs

In [22]:
def get_jinja_template():
    template_str = """
    {%- set _mode = mode | default('generate', true) -%}
    {%- set message_roles = ['SYSTEM', 'USER', 'ASSISTANT'] -%}
    {%- set ns = namespace(system_prompt='', system_prompt_handled=false, last_assistant_index_for_eos=-1, messages=messages) -%}
    {%- for message in ns.messages -%}
        {%- if not message.get('role') -%}
            {{ raise_exception('Key [role] is missing. Original input: ' +  message|tojson) }}
        {%- endif -%}
        {%- if message['role'] | upper not in message_roles -%}
            {{ raise_exception('Invalid role ' + message['role']|tojson + '. Only ' + message_roles|tojson + ' are supported.') }}
        {%- endif -%}
        {%- if not message.get('content') -%}
            {{ raise_exception('Key [content] is missing. Original input: ' +  message|tojson) }}
        {%- endif -%}
        {%- if loop.last and message['role'] | upper == 'ASSISTANT' -%}
            {%- set ns.last_assistant_index_for_eos = loop.index0 -%}
        {%- endif -%}
    {%- endfor -%}
    {%- if _mode == 'generate' -%}
        {{ bos_token }}
    {%- endif -%}
    {%- for message in ns.messages -%}
        {%- if message['role'] | upper == 'SYSTEM' and ns.system_prompt == '' -%}
            {%- set ns.system_prompt = message['content'] -%}
        {%- elif message['role'] | upper != 'SYSTEM' -%}
            {%- if _mode == 'train' and (message['role'] | upper == 'USER') != ((loop.index0 - (1 if ns.system_prompt != '' else 0)) % 2 == 0) -%}
                {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}
            {%- endif -%}
            {%- if message['role'] | upper == 'USER' and ns.system_prompt != '' and not ns.system_prompt_handled -%}
                {{- '[INST] ' + ns.system_prompt + ' ' + message['content'] + ' [/INST]' }}
                {%- set ns.system_prompt_handled = true -%}
            {%- elif message['role'] | upper == 'USER' -%}
                {{- '[INST] ' + message['content'] + ' [/INST]' }}
            {%- elif message['role'] | upper == 'ASSISTANT' -%}
                {%- if _mode == 'train' -%}
                    {{ unk_token + message['content'] + eos_token + unk_token }}
                {%- else -%}
                    {{- message['content'] + (eos_token if loop.index0 != ns.last_assistant_index_for_eos else '') }}
                {%- endif -%}
            {%- else -%}
                {{ raise_exception('Unexpected role found') }}
            {%- endif -%}
        {%- endif -%}
    {%- endfor -%}
    """

    return template_str

In [23]:
def fine_tune_model(dataset_name):
    """
    This function triggers the fine-tuning job for the selected dataset.

    Args:
        dataset_name (str): The dataset name (e.g., "care", "fashion", "outdoor", "baby")
    """
    # Retrieve the unique dataset ID associated with the given dataset name
    dataset_Id = DATASET_IDS[dataset_name]

    # Retrieve the unique model ID associated with the given dataset name
    model_Id = MODEL_IDS[dataset_name]

    # Define the payload for fine-tuning the model
    payload = {
        "displayName": f"Fine-tuning job for {dataset_Id}",
        "dataset": f"accounts/{ACCOUNT_ID}/datasets/{dataset_Id}",
        "baseModel": f"accounts/{ACCOUNT_ID}/models/{BASE_MODEL_ID}",
        "modelId": f"{model_Id}",
        "conversation": {
            "jinjaTemplate": get_jinja_template()
        }
    }

    # Fireworks API URL for fine-tuning job creation
    url = f"{BASE_URL}/fineTuningJobs"

    # Send the fine-tuning request
    response = requests.post(url, json=payload, headers=HEADERS_WITH_CONTENT_TYPE)

    # Handle response
    if response.status_code == 200:
      print(f"Fine-tuning job for {dataset_Id} created successfully.")
      print(response.json())
    else:
      print(f"Failed to create fine-tuning job for {dataset_Id}: {response.status_code}")
      print(response.text)
      response.raise_for_status()

In [24]:
# Loop through the datasets and process each one
for dataset_name in DATASETS:

  # Fine-tune for each specific dataset
  fine_tune_model(dataset_name)

Fine-tuning job for care-multilora-demo-test-data created successfully.
{'baseModel': 'accounts/fireworks/models/llama-v3p2-1b-instruct', 'batchSize': 32, 'containerVersion': '', 'conversation': {'jinjaTemplate': "\n    {%- set _mode = mode | default('generate', true) -%}\n    {%- set message_roles = ['SYSTEM', 'USER', 'ASSISTANT'] -%}\n    {%- set ns = namespace(system_prompt='', system_prompt_handled=false, last_assistant_index_for_eos=-1, messages=messages) -%}\n    {%- for message in ns.messages -%}\n        {%- if not message.get('role') -%}\n            {{ raise_exception('Key [role] is missing. Original input: ' +  message|tojson) }}\n        {%- endif -%}\n        {%- if message['role'] | upper not in message_roles -%}\n            {{ raise_exception('Invalid role ' + message['role']|tojson + '. Only ' + message_roles|tojson + ' are supported.') }}\n        {%- endif -%}\n        {%- if not message.get('content') -%}\n            {{ raise_exception('Key [content] is missing. Or

## Get Fine-Tuning Jobs IDs

In [20]:
def get_fine_tuning_job_Ids():
    """
    Updates the fine_tuning_jobs dictionary with fine-tuning job IDs
    by fetching and mapping them to their respective model IDs.
    """
    # Fireworks API URL to retrieve fine-tuning jobs
    url = f"{BASE_URL}/fineTuningJobs"

    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        fine_tuning_data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching fine-tuning jobs: {e}")
        return

    # Iterate through the retrieved jobs and map job IDs to model IDs
    for job in fine_tuning_data.get("fineTuningJobs", []):
        model_Id = job.get("modelId")
        fine_tuning_job_Id = job.get("name", "").split("/")[-1]

        # Update the corresponding entry in fine_tuning_jobs if a match is found
        for key, target_model_Id in MODEL_IDS.items():
            if target_model_Id == model_Id:
                fine_tuning_jobs[key] = fine_tuning_job_Id

    print("Updated fine_tuning_jobs:", fine_tuning_jobs)

In [21]:
get_fine_tuning_job_Ids()

Updated fine_tuning_jobs: {'care': '9c9769f23d894e8a86a00646c226ae45', 'fashion': 'f407f4c6dbe240b9a4f4a6680eeaa5a2', 'outdoor': '051c11ecee3d49fda7c578c4027abccf', 'baby': '0a22f519a0014760b92a34b72c8add58'}


## Validate Fine-Tuning Jobs

In [31]:
def validate_finetuning_job(dataset_name):
    """
    Validate the fine-tuning job on Fireworks.

    Args:
        dataset_name (str): Name of the dataset to validate.
    """
    # Retrieve the unique fine-tuning job ID associated with the given dataset name
    fine_tuning_job_id = fine_tuning_jobs[dataset_name]

    # Define the validation URL
    url = f"{BASE_URL}/fineTuningJobs/{fine_tuning_job_id}"

    # Make the GET request to validate the fine-tuning job
    response = requests.get(url, headers=HEADERS)

    # Handle the API response
    if response.status_code == 200:
        # Check that the status is "COMPLETED"
        dataset_dict = json.loads(response.text)
        state = dataset_dict["state"]
        print(f"Fine-tuning job {fine_tuning_job_id} is {state}.")
        print(response.json())
    else:
        print(f"Failed to complete fine-tuning job {fine_tuning_job_id}: {response.status_code}")
        print(response.text)

In [33]:
# Loop through the fine-tuning jobs and process each one
for dataset_name in fine_tuning_jobs:

  # Validate each fine-tuning job
  validate_finetuning_job(dataset_name)

Fine-tuning job 9c9769f23d894e8a86a00646c226ae45 is COMPLETED.
{'baseModel': 'accounts/fireworks/models/llama-v3p2-1b-instruct', 'batchSize': 32, 'containerVersion': '', 'conversation': {'jinjaTemplate': "\n    {%- set _mode = mode | default('generate', true) -%}\n    {%- set message_roles = ['SYSTEM', 'USER', 'ASSISTANT'] -%}\n    {%- set ns = namespace(system_prompt='', system_prompt_handled=false, last_assistant_index_for_eos=-1, messages=messages) -%}\n    {%- for message in ns.messages -%}\n        {%- if not message.get('role') -%}\n            {{ raise_exception('Key [role] is missing. Original input: ' +  message|tojson) }}\n        {%- endif -%}\n        {%- if message['role'] | upper not in message_roles -%}\n            {{ raise_exception('Invalid role ' + message['role']|tojson + '. Only ' + message_roles|tojson + ' are supported.') }}\n        {%- endif -%}\n        {%- if not message.get('content') -%}\n            {{ raise_exception('Key [content] is missing. Original in

# **Deploy and Use Fine-Tuned Models for Inference**

## Access and Download Base Model

### List available models

View all models in your Fireworks account:

`firectl list models`

Example output:

```
Code Llama 13B (code-llama-13b)    2024-02-29 20:36:24    HF_BASE_MODEL
CodeGemma 7B (codegemma-7b)        2024-06-19 22:57:22    HF_BASE_MODEL
...                                ...                    ...

```

Recall the supported base models:
* Gemma
* Phi, Phi-3
* Llama 1, 2, 3, 3.1
* LLaVa
* Mistral & Mixtral
* Qwen2
* StableLM
* Starcoder (GPTBigCode) & Starcoder2
* DeepSeek V1 & V2
* GPT NeoX

## Download base model

Download your chosen model to a local directory:

`firectl download model <model-id> <output-directory>`

Example:

`firectl download model code-llama-13b-instruct ./base_model`

Available flags:
* `--quiet`: Suppress progress bar
*  `-h`, `--help`: Display help information

## Obtain LoRA Adapters

### Download LoRA adapter from Fireworks

The fine-tuning jobs will have created PEFT add-ons on your Fireworks account. These LoRA adapters are displayed alongside models when you use the `firectl list models` command and are labeled with the type `HF_PEFT_ADDON`. To download a LoRA adapter, use the same command you would use to download a model

Example:

`firectl download model care-multilora-demo-test-model ./care_lora_adapter`

`firectl download model fashion-multilora-demo-test-model ./fashion_lora_adapter`

`firectl download model outdoor-multilora-demo-test-model ./outdoor_lora_adapter`

`firectl download model baby-multilora-demo-test-model ./baby_lora_adapter`

## Merge Base Model with LoRA Adapters

### Installation requirements

First, ensure you have the necessary libraries installed:

`pip install torch transformers peft`

### Merging script

Create a Python script (`merge_model.py`) with the following code:

```
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

def merge_lora_with_base_model(base_model_path: str, output_path: str):
    """
    Merge a LoRA adapter with a base model and save the result.

    Args:
        base_model_path (str): Path to the base model directory.
        output_path (str): Directory to save the merged model.
    """
    # Load the base model
    print(f"Loading base model from '{base_model_path}'...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # Load and apply LoRA adapters
    print("Loading LoRA adapters...")
    model = PeftModel.from_pretrained(
        model=base_model,
        model_id="./care_lora_adapter",
        adapter_name="care_lora_adapter"
    )
    model.load_adapter(
        model_id="./fashion_lora_adapter",
        adapter_name="fashion_lora_adapter"
    )
    model.load_adapter(
        model_id="./outdoor_lora_adapter",
        adapter_name="outdoor_lora_adapter"
    )
    model.load_adapter(
        model_id="./baby_lora_adapter",
        adapter_name="baby_lora_adapter"
    )

    # Merge adapters with the base model
    print("Merging LoRA adapters with the base model...")
    merged_model = model.merge_and_unload()

    # Save the merged model
    print(f"Saving merged model to '{output_path}'...")
    merged_model.save_pretrained(output_path)

    # Save the tokenizer
    print("Saving tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    tokenizer.save_pretrained(output_path)

    print("Merge completed successfully!")

if __name__ == "__main__":
    # Example usage
    merge_lora_with_base_model(
        base_model_path="base_model/hf",  # Directory containing the base model
        output_path="./final_merged_model"  # Output directory for the merged model
    )

  ```

### Running the merge

Execute the script after setting your paths:

`python merge_model.py`

Important: After merging, verify that all necessary tokenizer files are present in the output directory. The merging process might skip some essential tokenizer files. You may need to manually copy these files from the base model:
* tokenizer_config.json
* tokenizer.json
* special_tokens_map.json

These files can be found in the original base model directory!

## Upload and Deploy Merged Model

### Create model in Fireworks

Upload your merged model to Fireworks:

`firectl create model <model-name> <path/to/merged/model>`

Example:

`firectl create model mutlilora-demo-test-model ./final_merged_model`

For additional options:

`firectl create model -h`

### Create deployment

Deploy your uploaded model:

Basic deployment:

`firectl create deployment <model-name>`

Using full model path:

`firectl create deployment accounts/<account-name>/models/<model-name>`

Example:

```
firectl create deployment mutlilora-demo-test-model
# OR
firectl create deployment accounts/myaccount/models/mutli-lora-demo-model
```

Recall, for additional deployment parameters/configuration options:

`firectl create deployment -h`

### Verification

After deployment, you can verify the status using:

`firectl list deployments`

## Inference

In [9]:
def chat_completion_care_dataset():
    """
    Fetches the response for a user query related to the care dataset.
    """
    # API payload
    payload = {
        "model": "accounts/fireworks/models/mutlilora-demo-test-model",
        "messages": [
            {
                "role": "user",
                "content": "Give me the best-rated hair serum for curly hair to reduce frizz."
            }
        ]
    }

    # API endpoint URL
    url = "https://api.fireworks.ai/inference/v1/chat/completions"

    # Make a POST request to the Fireworks API
    response = requests.post(url, headers=HEADERS_WITH_CONTENT_TYPE, data=json.dumps(payload))

    if response.status_code == 200:
        response_data = response.json()
        message_content = response_data["choices"][0]["message"]["content"]
        print("Response from the care dataset:")
        print(message_content)
    else:
        print(f"Failed to get response for the query. Status code: {response.status_code}")
        print(response.text)

In [10]:
chat_completion_care_dataset()

Response from the care dataset:
Here are some of the best-rated hair serums for curly hair that can help reduce frizz:

1. **Moroccanoil Curl Defining Serum**: This lightweight, oil-based serum is formulated with antioxidants and argan oil to smooth and define curls while reducing frizz. (Amazon rating: 4.5/5)
	* Price: $38
2. **Bounce Curl Lightweight Hydrating Hair Serum**: This serum is infused with argan oil, coconut oil, and shea butter to provide long-lasting hydration and reduce frizz for curly hair. (Amazon rating: 4.5/5)
	* Price: $24
3. **Ouidad Curl Repair Moisturizing Serum**: This rich, oil-based serum is designed to nourish and hydrate dry, damaged curls while reducing frizz and flyaways. (Amazon rating: 4.5/5)
	* Price: $38
4. **Aunt Jackie's Quench Moisture Intensive Leave-In Conditioner Cream**: This rich, creamy leave-in conditioner is formulated with intense moisture to help lock in hydration and reduce frizz for curly hair. (Amazon rating: 4.5/5)
	* Price: $25
5. **

In [11]:
def chat_completion_fashion_dataset():
    """
    Fetches the response for a user query related to the fashion dataset.
    """
    # API payload
    payload = {
        "model": "accounts/fireworks/models/mutlilora-demo-test-model",
        "messages": [
            {
                "role": "user",
                "content": "Give me the best-rated women's casual shirt that is 100% cotton, can be washed in machine, and is wrinkle free."
            }
        ]
    }

    # API endpoint URL
    url = "https://api.fireworks.ai/inference/v1/chat/completions"

    # Make a POST request to the Fireworks API
    response = requests.post(url, headers=HEADERS_WITH_CONTENT_TYPE, data=json.dumps(payload))

    if response.status_code == 200:
        response_data = response.json()
        message_content = response_data["choices"][0]["message"]["content"]
        print("Response from the fashion dataset:")
        print(message_content)
    else:
        print(f"Failed to get response for the query. Status code: {response.status_code}")
        print(response.text)

In [12]:
chat_completion_fashion_dataset()

Response from the fashion dataset:
After researching and analyzing reviews, I found some great options for women's casual shirts that fit your criteria. Here are some top-rated, 100% cotton shirts that can be machine washed and come wrinkle-free:

**Option 1:**

*   **Cotton & Company Women's Soft White Cotton Shirt**: This shirt is made from 100% cotton, is wrinkle-free, and can be machine washed. It's also quite comfortable to wear, with a relaxed fit that's perfect for everyday wear. Cotton & Company is a popular brand that offers high-quality, cotton-based clothing.
*   **Price**: Around $20-$30
*   **Rating**: 4.5/5 stars on Amazon (buy now)

**Option 2:**

*   **Patagonia Women's Cotton & Linen Shrug**: While Patagonia is a bit pricier, their cotton & linen blends are renowned for their durability and comfort. This specific shirt is made from 100% cotton and linen, making it wrinkle-free and machine washable. The extra-long sleeve provides coverage and warmth for casual wear, whi

In [13]:
def chat_completion_outdoor_dataset():
    """
    Fetches the response for a user query related to the outdoor dataset.
    """
    # API payload
    payload = {
        "model": "accounts/fireworks/models/mutlilora-demo-test-model",
        "messages": [
            {
                "role": "user",
                "content": "Give me the best-rated outdoor golf club that is less than 8 pounds in weight."
            }
        ]
    }

    # API endpoint URL
    url = "https://api.fireworks.ai/inference/v1/chat/completions"

    # Make a POST request to the Fireworks API
    response = requests.post(url, headers=HEADERS_WITH_CONTENT_TYPE, data=json.dumps(payload))

    if response.status_code == 200:
        response_data = response.json()
        message_content = response_data["choices"][0]["message"]["content"]
        print("Response from the outdoor dataset:")
        print(message_content)
    else:
        print(f"Failed to get response for the query. Status code: {response.status_code}")
        print(response.text)

In [15]:
chat_completion_outdoor_dataset()

Response from the outdoor dataset:
Finding the best outdoor golf clubs that meet your specific needs and weight requirements can be a challenge. However, I'd be happy to provide you with some insights and recommendations that might satisfy your requirements.

**Based on current market research and reviews, here are some top outdoor golf clubs that weigh under 8 pounds:**

1. **TaylorMade Golf Flex Drive Putter Set (8 oz)**: Weight: 7.8 lbs. This putter set is designed for golfers of all skill levels and features a graphite shaft, which is optimized for forgiveness and speed.
	* Average customer review rating: 4.7/5
2. **Mizuno Iron 823 UST Golf Putters (8.1 oz)**: Weight: 8.1 oz. These putters offer a great balance of distance and power, with a forgiving core and a UST shaft that provides quick and precise shots.
	* Average customer review rating: 4.6/5
3. **Ping G410i Men's Putter (7.7 oz)**: Weight: 7.7 oz. These putters feature a forgiving design and a high-launching core, making th

In [16]:
def chat_completion_baby_dataset():
    """
    Fetches the response for a user query related to the baby dataset.
    """
    # API payload
    payload = {
        "model": "accounts/fireworks/models/mutlilora-demo-test-model",
        "messages": [
            {
                "role": "user",
                "content": "Give me baby blanket that is best-rated, has a two-way zipper, and is made up of micro-fiber."
            }
        ]
    }

    # API endpoint URL
    url = "https://api.fireworks.ai/inference/v1/chat/completions"

    # Make a POST request to the Fireworks API
    response = requests.post(url, headers=HEADERS_WITH_CONTENT_TYPE, data=json.dumps(payload))

    if response.status_code == 200:
        response_data = response.json()
        message_content = response_data["choices"][0]["message"]["content"]
        print("Response from the baby dataset:")
        print(message_content)
    else:
        print(f"Failed to get response for the query. Status code: {response.status_code}")
        print(response.text)

In [17]:
chat_completion_baby_dataset()

Response from the baby dataset:
Here are some top-rated baby blanket options with a two-way zipper and made of micro-fiber:

1. **Ugg Adirondack Throw Micro-Medium Weight Blanket**:
	* Amazon rating: 4.7/5
	* Material: Micro-fiber
	* Features: Two-way zipper, compact weave for warmth, comes in 3 weights
2. **Moyenne by Moogly Micro-Fiber 2-in-1 Throw Blanket**:
	* Amazon rating: 4.9/5
	* Material: Micro-fiber
	* Features: Velcro closure for two-way use, available in 2 weights, perfect for snuggling up to baby
3. **SnugglePak Lightweight Micro-Fiber Baby Blanket**:
	* Amazon rating: 4.5/5
	* Material: Micro-fiber
	* Features: Two-way zipper, soft and lightweight, perfect for warm weather
4. **Orthopedic Micro-Fiber Blanket by Sensycloth**:
	* Amazon rating: 4.6/5
	* Material: Micro-fiber
	* Features: Orthopedic features, features a two-way zipper, and a soft, breathable fabric
5. **Ubevi Reversible Micro-Fiber Blanket**:
	* Amazon rating: 4.5/5
	* Material: Micro-fiber
	* Features: Reve

# **Clean up**

## Undeploy and Delete Deployment

### Undeploy Deployment

To undeploy a model on the Fireworks AI platform, you can use the firectl undeploy command. Here's how to use it:

`firectl undeploy <model-name> [flags]`

For example:

`firectl undeploy mutlilora-demo-test-model`

Available flags:
* `-h`, `--help`: Display help information for the undeploy command
* `--wait`: Wait until the model is undeployed

### Delete Model

To delete a model using the firectl command-line tool, you can use the following command:

`firectl delete model <model-name> [flags]`

For example:

`firectl delete model mutlilora-demo-test-model`

This command deletes a model from the Fireworks AI platform.

## Delete Fine Tuning Jobs

In [22]:
def delete_fine_tuning_job(dataset_name):
  """
  Deletes a fine-tuning job from Fireworks AI.

  Args:
      dataset_name (str): The name related to the fine-tuning job to be deleted.
  """
  # Retrieve the unique fine-tuning job ID associated with the given dataset name
  fine_tuning_job_id = fine_tuning_jobs[dataset_name]

  # Define the API endpoint for fine-tuning job deletion
  url = f"{BASE_URL}/fineTuningJobs/{fine_tuning_job_id}"

  # Make the DELETE request to remove the fine-tuning job
  response = requests.delete(url, headers=HEADERS)

   # Handle response
  if response.status_code == 200:
    print(f"Deleted fine-tuning job {fine_tuning_job_id} successfully.")
  else:
    print(f"Failed to delete fine-tuning job {fine_tuning_job_id}: {response.status_code}")
    print(response.text)
    response.raise_for_status()

In [23]:
# Loop through the fine-tuning jobs and process each one
for dataset_name in fine_tuning_jobs:

  # Delete each fine_tuning_job
  delete_fine_tuning_job(dataset_name)

Deleted fine-tuning job 9c9769f23d894e8a86a00646c226ae45 successfully.
Deleted fine-tuning job f407f4c6dbe240b9a4f4a6680eeaa5a2 successfully.
Deleted fine-tuning job 051c11ecee3d49fda7c578c4027abccf successfully.
Deleted fine-tuning job 0a22f519a0014760b92a34b72c8add58 successfully.


## Delete Datasets

In [24]:
def delete_dataset(dataset_name):
  """
  Deletes a dataset from Fireworks AI.

  Args:
      dataset_name (str): The name of the dataset to be deleted.
  """
  # Retrieve the unique dataset ID associated with the given dataset name
  dataset_Id = DATASET_IDS[dataset_name]

  # Define the API endpoint for dataset deletion
  url = f"{BASE_URL}/datasets/{dataset_Id}"

  # Make the DELETE request to remove the dataset
  response = requests.delete(url, headers=HEADERS)

   # Handle response
  if response.status_code == 200:
    print(f"Deleted dataset {dataset_Id} record successfully.")
  else:
    print(f"Failed to delete dataset {dataset_Id} record: {response.status_code}")
    print(response.text)
    response.raise_for_status()

In [25]:
# Loop through the datasets and process each one
for dataset_name in DATASETS:

  # Delete each dataset
  delete_dataset(dataset_name)

Deleted dataset care-multilora-demo-test-data record successfully.
Deleted dataset fashion-multilora-demo-test-data record successfully.
Deleted dataset outdoor-multilora-demo-test-data record successfully.
Deleted dataset baby-multilora-demo-test-data record successfully.
